In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
from sklearn.preprocessing import StandardScaler, Binarizer, LabelEncoder, Normalizer, OneHotEncoder

  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8913 sha256=e2c428c741b984dd14c7b5b35453caf4daf4d63c6a73325529cf597d7500e8d5
  Stored in directory: c:\users\roman studer\appdata\local\pip\cache\wheels\6e\df\38\abda47b884e3e25f9f9b6430e5ce44c47670758a50c0c51759
Successfully built et-xmlfile


# Preprocessing Pipeline, uveitis_data.xlsx

In [12]:
# import dataframe
df = pd.read_excel("../data/uveitis_data.xlsx")
assert len(df) >= 1075, "Data is not complete"
df.head()

,SUBJECT ID,Gender,Race,Location,Categorical,"Other (RV, Choroidal granuloma, pars planitis)",EHR DIAGNOSIS,Specific Diagnosis,notes,AC Abn Od Cells,...,Range34,Proteinase-3 Antibodies (Blood),UOM35,Range35,Rheumatoid Factor (Blood),UOM36,Range36,HBc (HepB core) Ab (Blood),HBs (HepB surface) Ag (Blood),HCV (HepC) Ab (Blood)
0,1,Male,Black/African American,Anterior,Nonneoplastic Masquerade,NaN,Posner-Schlossman ...,Idiopathic_anterior,NaN,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
1,2,Female,Black/African American,Intermediate,Idiopathic,NaN,Ant/Int Uveitis ...,pars planitis,pars planitis,2,...,<0.4 (Negative),<0.2,U,<0.4 (Negative),<10,IU/mL,<13,NEGATIVE,NEGATIVE,NEGATIVE
2,3,Female,White,Pan,Systemic,NaN,VKH,VKH,NaN,0.5,...,NaN,NaN,NaN,NaN,<10,IU/mL,<13,NEGATIVE,NEGATIVE,NEGATIVE
3,4,Female,White,Anterior,Idiopathic,NaN,chronic idiopathic anterior uveitis ...,Idiopathic_anterior,NaN,0,...,NaN,NaN,NaN,NaN,<15,IU/mL,<15,NEGATIVE,NEGATIVE,NEGATIVE
4,5,Female,Unknown Race,Anterior,Idiopathic,NaN,Acute Anterior Uveitis ...,Idiopathic_anterior,NaN,0,...,NaN,NaN,NaN,NaN,<10,IU/mL,<13,NEGATIVE,NEGATIVE,NEGATIVE
